## Image Segmentation for Medical diagnosis using PyTorch

In [191]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.checkpoint import checkpoint
from torch import optim
from torch.utils.data import random_split
from torchvision.transforms import transforms
from torchinfo import summary
from collections import defaultdict
import matplotlib.pyplot as plt
import cv2
import glob
import json
import pathlib
from time import time
from tqdm import tqdm

### Downlaod the CVC clinic dataset and extract

In [192]:
# !curl -L -o ~/Downloads/cvcclinicdb.zip\
#   https://www.kaggle.com/api/v1/datasets/download/balraj98/cvcclinicdb
# !mkdir data
# !unzip ~/Downloads/cvcclinicdb.zip -d data

In [193]:
with open("./config.json","r") as f:
    config = json.load(f)
config
image_path = config['data_path']+"Original/"
mask_path  = config['data_path']+"Ground Truth/"
images = glob.glob(image_path+"*"+config['file_extn'])
masks  = glob.glob(mask_path+"*"+config['file_extn'])

## Create CVC custom dataset and pre process

In [194]:
"""
Make CVC dataset
"""

class CVCDataset(Dataset):
    def __init__(self, images, image_transforms=None, mask_transforms=None):
        super(CVCDataset, self).__init__()
        self.images = images
        self.image_transforms = image_transforms
        self.mask_transforms = mask_transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img_split = img_path.split('/')
        img_name = img_split[-1]
        img_dir = '/'.join(img_split[:-2])
        mask_path = img_dir+"/Ground Truth/"+img_name
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if self.image_transforms:
            image = self.image_transforms(image)
        if self.mask_transforms:
            mask = self.mask_transforms(mask)
        # inv_mask = 1 - mask
        # mask = torch.concat([inv_mask, mask], dim=0)
        return image, mask


### Create train and test set

In [195]:
# Custom transform to threshold the mask

class ThresholdTransform(object):
  def __init__(self, thr=0.5):
    self.thr = thr

  def __call__(self, x):
    return (x > self.thr).to(x.dtype)  # do not change the data type


image_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    #transforms.Normalize()
])

mask_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    ThresholdTransform()
])

imgset = CVCDataset(images, image_transforms, mask_transforms)
trainset, testset = random_split(imgset, [0.9, 0.1])
batch_size = 32
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(testset, batch_size=8, shuffle=True, drop_last=True)

### Check image and mask sample

In [196]:
data, mask = next(iter(train_loader))
data = torch.einsum("chw->hwc", data[0])
mask = torch.einsum("chw->hwc", mask[0])
# mask.shape

In [ ]:
plt.imshow(data)

In [ ]:
plt.imshow(mask, cmap='gray')

In [199]:
""" Parts of the U-Net model """

class DoubleConv(nn.Module):
    """(conv => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super(DoubleConv, self).__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """ Downscaling with maxpool """

    def __init__(self):
        super(Down,self).__init__()
        self.down = nn.MaxPool2d(2)
    
    def forward(self, x):
        return self.down(x)


class DownDoubleConv(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super(DownDoubleConv, self).__init__()
        self.down_doubleconv = nn.Sequential(
            Down(),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.down_doubleconv(x)


class UpDoubleConv(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super(UpDoubleConv, self).__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.double_conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.double_conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is BCHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.double_conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [200]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False, checkpointing=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        self.checkpointing = checkpointing

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (DownDoubleConv(64, 128))
        self.down2 = (DownDoubleConv(128, 256))
        self.down3 = (DownDoubleConv(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (DownDoubleConv(512, 1024 // factor))
        self.up1 = (UpDoubleConv(1024, 512 // factor, bilinear))
        self.up2 = (UpDoubleConv(512, 256 // factor, bilinear))
        self.up3 = (UpDoubleConv(256, 128 // factor, bilinear))
        self.up4 = (UpDoubleConv(128, 64, bilinear))
        self.outc = (OutConv(64, n_classes))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        if self.checkpointing:
            x1 = checkpoint(self.inc, x, use_reentrant=False)
            x2 = checkpoint(self.down1, x1, use_reentrant=False)
            x3 = checkpoint(self.down2, x2, use_reentrant=False)
            x4 = checkpoint(self.down3, x3, use_reentrant=False)
            x5 = checkpoint(self.down4, x4, use_reentrant=False)
            x = checkpoint(self.up1, x5,x4, use_reentrant=False)
            x = checkpoint(self.up2, x,x3, use_reentrant=False)
            x = checkpoint(self.up3, x,x2, use_reentrant=False)
            x = checkpoint(self.up4, x,x1, use_reentrant=False)
            logits = checkpoint(self.outc, x, use_reentrant=False)
            # probs = checkpoint(self.sigmoid, logits, use_reentrant=False)
            return logits

        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


In [201]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# test_input, ground_truth = next(iter(train_loader))
# test_input = test_input.to(device)
# # test_input = torch.einsum("bchw->bhwc", test_input)
# ground_truth = torch.einsum("bchw->bhwc", ground_truth)
# plt.imshow(ground_truth[0], cmap='gray')
# test_model = UNet(3,1,checkpointing=True).to(device)
# test_output = test_model(test_input)
# test_output = torch.einsum("bchw->bhwc", test_output.detach().cpu())
# test_output_bin = (test_output > 0.5).to(test_output.dtype)
# plt.imshow(test_output[0], cmap='gray')
# plt.imshow(test_output_bin[0], cmap='gray')

In [202]:
# # test model
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# test_input = torch.randn((16, 3, 256, 256)).to(device)
# test_model = UNet(3,2,checkpointing=True).to(device)
# print(summary(test_model))
# torch.cuda.reset_peak_memory_stats()
# test_out = test_model(test_input)
# print(f"Peak Memory Usage With Checkpointing: {torch.cuda.max_memory_allocated()/1e6} MB")
# print(test_out.shape)

In [203]:
def save_plots(losses, name=None):
    pathlib.Path("results").mkdir(parents=True, exist_ok=True)
    if not name:
        timestr = time.strftime("%Y%m%d-%H%M%S")
        save_plot_path = "results/plot-"+timestr
    else:
        save_plot_path = "results/"+name
    epochs = len(losses['train_loss'])
    plt.plot(range(1,epochs+1), losses['train_loss'], label='Train loss')
    plt.plot(range(1,epochs+1), losses['val_loss'], label='Validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f"Losses")
    plt.savefig(save_plot_path, facecolor='w', edgecolor='none')
    plt.show()

def save_model(model, name=None):
    pathlib.Path("results/saved_models").mkdir(parents=True, exist_ok=True)
    if not name:
        timestr = time.strftime("%Y%m%d-%H%M%S")
        save_model_path = "results/saved_models/"+timestr+".pth"
    else:
        save_model_path = "results/saved_models/"+name+".pth"
    torch.save(model.state_dict(), save_model_path)

### Define custom IOU metric

In [204]:
# IoU metric

def custom_iou(predictions: torch.tensor, targets: torch.tensor):
    # inputs are of shape BxCxHxW
    assert isinstance(predictions, torch.Tensor), "Not a torch tensor"
    assert isinstance(targets, torch.Tensor), "Not a torch tensor"
    smooth = 1e-6
    n_channels = predictions.shape[1]
    predictions = F.sigmoid(predictions) if n_channels == 1 else F.softmax(predictions, dim=1)
    # intersection : element wise multiplication or logical AND
    intersection = torch.logical_and(predictions, targets).float().sum((1,2,3))
    # union : element wise sum or logical OR
    union = torch.logical_or(predictions, targets).float().sum((1,2,3))
    iou = (intersection + smooth) / (union + smooth)
    return iou.mean()

"""
dice metric
dice_coeff = [2 * intersection(A,B)] / [sum of areas of A and B]
Intersection can be expressed as multiplication. Dice coeff is differentiable and
can be part of the loss function.
""" 
def custom_dice(predictions: torch.tensor, targets: torch.tensor):
    # inputs are of shape BxCxHxW
    # return dice loss, to be used in loss function
    assert isinstance(predictions, torch.Tensor), "Not a torch tensor"
    assert isinstance(targets, torch.Tensor), "Not a torch tensor"
    smooth = 1e-6
    n_channels = predictions.shape[1]
    predictions = F.sigmoid(predictions) if n_channels == 1 else F.softmax(predictions, dim=1)
    intersection = (2 * predictions * targets).float().sum((1,2,3))
    total = predictions.sum((1,2,3)) + targets.sum((1,2,3))
    dice_coeffs = (intersection + smooth) / (total + smooth)
    return 1 - dice_coeffs.mean()


In [205]:
def train_model(model, device, train_loader, loss_criterion, optimizer, epoch):
    model.train()
    train_loss = 0
    total = 0
    total_iou = 0
    n_train = len(train_loader.dataset)
    with tqdm(total=n_train, desc=f'Epoch {epoch}', unit='img') as pbar:
        for batchidx, (data, target) in enumerate(train_loader):
            #print(f'batch id: {batchidx}')
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            grad_scaler = torch.amp.GradScaler(enabled=True)
            with torch.autocast(device_type=device.type):
                output = model(data)
                loss = loss_criterion(output, target)
                loss += custom_dice(output, target)
                train_loss += loss.item()
            grad_scaler.scale(loss).backward() # loss.backward()
            grad_scaler.step(optimizer) # optimizer.step()
            grad_scaler.update()
            predictions = nn.functional.sigmoid(output)
            predictions = (predictions > 0.5).to(predictions.dtype)
            iou = custom_iou(output, target)
            total_iou += iou.item()
            total += len(data)
            pbar.update(len(data))
        avg_iou = total_iou/len(train_loader)
        avg_loss = train_loss/total
        pbar.set_postfix({'avg_loss (epoch)': avg_loss, 'IoU': avg_iou})
        # print(f"Train epoch: {epoch} [{(batchidx+1) * len(data)}/{len(train_loader.dataset)}\
        #   ({(100. * batchidx/len(train_loader)):.0f}%)]\tLoss: {avg_loss:.6f}\tIoU: {avg_iou:.6f}")
    return avg_iou, avg_loss

def test_model(model, device, test_loader, loss_criterion):
    model.eval()
    test_loss = 0.
    total_iou = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_criterion(output, target).item()
            predictions = nn.functional.sigmoid(output)
            predictions = (output > 0.5).to(predictions.dtype)
            iou = custom_iou(output, target)
            total_iou += iou.item()
            total += len(data)
    test_loss /= total
    test_iou = total_iou/len(test_loader)
    print(f"Test set: Average loss: {test_loss:.4f}, IoU: {test_iou:.6f})")
    return test_iou, test_loss


In [ ]:
num_classes = 1
learning_rate = 1e-3
num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_criterion = nn.BCEWithLogitsLoss() # nn.BCELoss()
inp_channels = 3
model = UNet(inp_channels, num_classes, checkpointing=True).to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

losses = defaultdict(list)

print(f"using device: {device}")
# print(summary(model))
n_train = len(train_loader.dataset)
for epoch in range(1,num_epochs+1):
    _, tloss = train_model(model, device, train_loader, loss_criterion, optimizer, epoch)
    losses['train_loss'].append(tloss)
    _, vloss = test_model(model, device, test_loader, loss_criterion)
    losses['val_loss'].append(vloss)

# save plots
save_plots(losses, 'mnist-loss')

#save model
save_model(model, 'mnist_sample_model')

In [ ]:
test_input, ground_truth = next(iter(train_loader))
test_input = test_input.to(device)
# ground_truth = torch.einsum("bchw->bhwc", ground_truth)
test_output = model(test_input)
test_ip = torch.einsum("bchw->bhwc", test_input).detach().cpu()
test_output = nn.functional.sigmoid(test_output)
# test_output_bg = test_output[:,0,:,:].to(test_output.dtype)
test_output_tumor = test_output[:,0,:,:].to(test_output.dtype)
# test_output = torch.einsum("bchw->bhwc", test_output.detach().cpu())
# test_output_bg = test_output_bg.detach().cpu()
test_output_tumor = test_output_tumor.detach().cpu()

fig, axs = plt.subplots(1,3)
idx = 10
axs[0].imshow(test_ip[idx])
axs[1].imshow(ground_truth[idx,0,:,:], cmap='gray')
axs[2].imshow(test_output_tumor[idx,:,:], cmap='gray')
plt.show()